# NPCC Power Flow Tunning

Reference:

1. [Power Grid Lib - Optimal Power Flow, https://github.com/power-grid-lib/pglib-opf](https://ltb.readthedocs.io/projects/ams/en/latest/getting_started/formats/matpower.html)
1. [WECC-and-NPCC-Electricity-Economic-Data](https://github.com/enliten/ENLITEN-Grid-Econ-Data)

In [1]:
import numpy as np
import pandas as pd
import scipy

import andes
import ams

import datetime

In [2]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f'andes:{andes.__version__}')
print(f'ams:{ams.__version__}')

Last run time: 2024-04-08 22:59:23
andes:1.9.2
ams:0.9.5.post25+g738d558


In [3]:
andes.config_logger(stream_level=30)
ams.config_logger(stream_level=20)

In [4]:
npcc_load_src = scipy.io.loadmat('./data/NPCC_load.mat')
npcc_load_data = npcc_load_src['load_dis']

In [5]:
# read m file to mpc
mpc = andes.io.matpower.m2mpc('./data/NPCC.m')

for n in range(30):
    # adjust the load
    mpc['bus'][:, 2] = npcc_load_data[:, n]

    # initialize an empty system
    sa = andes.system.System(setup=False, no_output=True, default_config=True)

    # read into an ANDES system
    andes.io.matpower.mpc2system(mpc, sa)

    sa.setup()

    pflow_res = sa.PFlow.run()

    underV = sa.Bus.v.v < sa.Bus.vmin.v
    overV = sa.Bus.v.v > sa.Bus.vmax.v
    outV = underV | overV

    p0 = sa.PQ.p0.v.sum()
    q0 = sa.PQ.q0.v.sum()
    print(f'Hour {n}: P={p0:.4f}, Q={q0:.4f}; PFlow converged: {sa.PFlow.converged}; V out of range: {outV.any()}')

Hour 0: P=298.9270, Q=40.6650; PFlow converged: True; V out of range: False
Hour 1: P=292.3349, Q=40.6650; PFlow converged: True; V out of range: False
Hour 2: P=285.8032, Q=40.6650; PFlow converged: True; V out of range: False
Hour 3: P=279.3979, Q=40.6650; PFlow converged: True; V out of range: False
Hour 4: P=272.7098, Q=40.6650; PFlow converged: True; V out of range: False
Hour 5: P=266.5994, Q=40.6650; PFlow converged: True; V out of range: False
Hour 6: P=261.2480, Q=40.6650; PFlow converged: True; V out of range: False
Hour 7: P=256.7017, Q=40.6650; PFlow converged: True; V out of range: False
Hour 8: P=253.4065, Q=40.6650; PFlow converged: True; V out of range: False
Hour 9: P=252.2642, Q=40.6650; PFlow converged: True; V out of range: False
Hour 10: P=252.6824, Q=40.6650; PFlow converged: True; V out of range: False
Hour 11: P=254.9175, Q=40.6650; PFlow converged: True; V out of range: False
Hour 12: P=257.8131, Q=40.6650; PFlow converged: True; V out of range: False
Hour 13: 